In [ ]:
# %pip install hdbscan
# %pip install hdbscan-with-cosine-distance
# %pip install gmsdb

In [22]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# Загрузка модели
model = SentenceTransformer("./sentence_bert_service_desk_model")

# Новые сообщения для тестирования
new_questions = ["Директум Коллеги, добрый день, \r\n\r\n \r\n\r\nПрошу разблокировать учетную запись.\r\n\r\n \r\n\r\n\r\n"]

# Преобразование вопросов в векторы
question_embeddings = model.encode(new_questions)

# Пример ответов (можно заменить на реальные данные)
df = pd.read_excel("TEMPLATES.xlsx")
new_answers = df['Текст'].tolist()

# Преобразование ответов в векторы
answer_embeddings = model.encode(new_answers)

# Теперь вы можете использовать cosine similarity или другую меру для оценки схожести между вопросами и ответами.
from sklearn.metrics.pairwise import cosine_similarity

# Вычисление схожести
similarity_matrix = cosine_similarity(question_embeddings, answer_embeddings)

m, im = 0, "err"
# Вывод результатов
for i, question in enumerate(new_questions):
    question = question.replace("\r\n", " ")
    print(f"Вопрос: {question}")
    for j, answer in enumerate(new_answers):
        temp = similarity_matrix[i][j]
        print(f"  Ответ: {answer} - Сходство: {temp:.4f}")
        if temp > m:
            m, im = temp, answer

print(im)

Вопрос: Директум Коллеги, добрый день,      Прошу разблокировать учетную запись.      
  Ответ: Для выдачи нового оборудования вам необходимо оформить обращение по ссылке: - Сходство: 0.1379
  Ответ: Для получения SIM-карты и других манипуляций, касающихся мобильной связи, заполните заявку по ссылке: - Сходство: 0.1504
  Ответ: Для получения поддержки по программному обеспечению заполните заявку по ссылке: - Сходство: 0.2005
  Ответ: Для установки драйверов для МФУ заполните заявку по ссылке: - Сходство: 0.0596
  Ответ: Для передачи оборудования оформите заявку по ссылке: - Сходство: 0.1167
  Ответ: Для решения проблемы с IT-оборудованием обратитесь по ссылке: - Сходство: 0.2426
  Ответ: Для бронирования временного оборудования заполните заявку по ссылке: - Сходство: 0.1885
  Ответ: Для организации поддержки мероприятия заполните заявку по ссылке: - Сходство: 0.2481
  Ответ: Для обслуживания принтера заполните заявку по ссылке: - Сходство: 0.1120
  Ответ: Для получения доступа к Консул

In [6]:
similarity_matrix[0]

array([0.2322748 , 0.18986107, 0.41250625, 0.42545882, 0.24567792,
       0.29717457, 0.23398831, 0.34684938, 0.28971964, 0.27187416,
       0.29851446, 0.13433036, 0.4615872 , 0.19820532, 0.1431424 ,
       0.1616571 , 0.13558894, 0.15938291], dtype=float32)

In [2]:
query = "Здравствуйте, просьба сбросить пароль от моего аккаунта в SAP (ERP MONO) – остановилась работа. Оба ваших портала заблокированы, просьба принять запрос по почте."
query_embeddings = model.encode(query)

In [3]:
from tqdm import tqdm
import numpy as np

# Включаем tqdm для визуализации
def encode_with_progress(model, texts, batch_size=32):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Encoding texts", ncols=100, dynamic_ncols=True):
        batch = texts[i:i + batch_size]
        batch_embeddings = model.encode(batch, show_progress_bar=False)
        embeddings.extend(batch_embeddings)
    return np.array(embeddings)

a = pd.read_csv("answers.csv", encoding="utf-16")["1"]
n = 128
a = a[:n]
embeddings = encode_with_progress(model, [str(a[i]) if a[i] is not None else "" for i in range(n)])

Encoding texts: 100%|██████████| 4/4 [00:09<00:00,  2.34s/it]


In [ ]:
embeddings[0] = model.encode("Добрый день, попробуйте перейти по данной ссылке на it портал ELMA365 . В случае ее недоступности, примем запрос через почту. ")

In [4]:
from sentence_transformers import util

print(util.semantic_search(query_embeddings, embeddings)[0])

[{'corpus_id': 77, 'score': 0.7897090911865234}, {'corpus_id': 106, 'score': 0.7615180611610413}, {'corpus_id': 76, 'score': 0.7534220218658447}, {'corpus_id': 19, 'score': 0.7369711399078369}, {'corpus_id': 2, 'score': 0.6573807001113892}, {'corpus_id': 3, 'score': 0.6280403137207031}, {'corpus_id': 35, 'score': 0.6160039305686951}, {'corpus_id': 99, 'score': 0.5998386144638062}, {'corpus_id': 51, 'score': 0.5995620489120483}, {'corpus_id': 75, 'score': 0.5953553318977356}]


In [5]:
print(a[77])


Добрый день,Сброс пароля в локальном SAP можно запросить по данной форме.  


In [1]:
import pandas as pd

q_emb = pd.read_csv("q_embeddings.csv")

In [2]:
from tqdm import tqdm

X = []
for i in tqdm(range(29417)):
    temp = [0]*768
    for j in range(768):
        temp[j] = float(q_emb[str(j)][i])
    X.append(temp)

100%|██████████| 29417/29417 [00:58<00:00, 499.80it/s]


In [ ]:
import numpy as np
from gmsdb import GMSDB

clf = GMSDB(
    n_components=40,
    # min_components=2,
    # step_components=100,
    # rand_search=1000,
    # rand_level=0.5,
    verbose=True,
    show=True,
)
Yp = clf.fit_predict(np.array(X))

C: 2 bic: 634425.7093905134
C: 3 bic: 1467207.595406875
C: 4 bic: 3857304.921816244
C: 5 bic: 6218389.63595105
C: 6 bic: 8313516.690532186
C: 7 bic: 10639886.022891618
C: 8 bic: 13107782.002648056
C: 9 bic: 15557245.598458914
C: 10 bic: 17894967.88878322
C: 11 bic: 20301951.8923869
C: 12 bic: 22608656.940031737
C: 13 bic: 25061440.511142757
C: 14 bic: 27480563.040786147
C: 15 bic: 29528322.449963115
C: 16 bic: 31776134.14324575
C: 17 bic: 33777586.487431325
C: 18 bic: 36500375.41634917
C: 19 bic: 39436116.97129945
C: 20 bic: 40869137.91725217
C: 21 bic: 43094160.30816343
C: 22 bic: 45855415.96038725
C: 23 bic: 47441831.3417194
C: 24 bic: 49961973.4432474
C: 25 bic: 51318497.07164188
C: 26 bic: 53310173.69681119
C: 27 bic: 55611954.94114348
C: 28 bic: 55836973.77422718
C: 29 bic: 59047034.613239065
C: 30 bic: 59993657.29394579
C: 31 bic: 62292532.67820163
C: 32 bic: 63706447.46887613
C: 33 bic: 65579366.424692735
C: 34 bic: 66212087.45207137
C: 35 bic: 67367076.37599231
C: 36 bic: 67947

In [ ]:
print(len(clusters[clusters==-1]))
print(len(clusters[clusters==0]))
print(len(clusters[clusters==1]))

In [46]:
import pandas as pd

q = pd.read_csv("last_ye_q.csv", encoding="utf-16")
a = pd.read_csv("last_ye_a.csv", encoding="utf-16")

In [48]:
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# Загрузка модели
model = SentenceTransformer('./sentence_bert_service_desk_model')
q_embeddings = model.encode(q["0"], show_progress_bar=True)
a_embeddings = model.encode(a["1"], show_progress_bar=True)

Batches:   0%|          | 0/187 [00:00<?, ?it/s]

Batches:   0%|          | 0/187 [00:00<?, ?it/s]

In [57]:
from sentence_transformers import util

bag = []
qq = []
for i in tqdm(range(len(q_embeddings))):
    l = util.semantic_search(q_embeddings[i], a_embeddings)
    temp = l[0][0]['corpus_id']
    if temp not in bag:
        bag.append(temp)
        qq.append(q["0"][i])

100%|██████████| 5978/5978 [00:23<00:00, 254.79it/s]


In [58]:
qq[8], qq[11]

(' Ошибка в подключении Check Point\xa0Добрый день!\xa0Работаю из дома и выдает ошибку при подключении Check Point.Darya Fomina',
 ' check point issue\xa0Коллеги, приветНе могу совсем подключить чек-пойнт при работе из дома, и создать тикет нормально тоже не могу.Ну и работать тоже не могу, поскольку тимс меня не идентифицирует и все прочие проблемы – паровозом.\xa0Пыталась поставить другой сайт – стало еще хуже( \xa0HEEELP\xa0С уважением\xa0Мария Беляева Nestle Russia/ Supply ChainСustomer Facing Supply Chain ManagerCell phone +7\xa0916\xa0153-19-22mailto:Mariya.Belyaeva@ru.nestle.com\xa0')

In [61]:
bag[8], bag[11]

(4393, 5892)

In [76]:
d = {qq[i]: a["0"][bag[i]] for i in range(2015)}
df = pd.DataFrame(columns=["q", "a"], index=range(2015))
for i in range(2015):
    temp = list(d)[i]
    df["q"][i] = temp
    df["a"][i] = d[temp]
df.to_csv("q-a_last_ye.csv", index=False)

C:\Users\RUShulpiAn\AppData\Local\Temp\ipykernel_16464\1484029725.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["q"][i] = temp
C:\Users\RUShulpiAn\AppData\Local\Temp\ipykernel_16464\1484029725.py:6: FutureWarning: ChainedAssignmentE

In [77]:
df

,q,a
0,Об установке сертификата Webaccessagent ssl Д...,RE: Вопрос по работе веб-агента Directum
1,смена пароля Добрый день. Просьба помочь прой...,"RE: Не дает сменить пароль, сегодня последний ..."
2,Сброс пароля_КайзероваImportance: High Добрый...,RE: смена фамилии - Ткаченко на Авилову в ЦПБ ...
3,"RE: Новый консультант Карен Гумеров Коллеги, ...",RE: Запрос пароля Миронова Татьяна
4,"Доступ на диск N Добрый день, Прошу поделить...",RE: доступ к папке на диске N
...,...,...
2010,заблокирована учетная запись у USER ID RUBOYT...,RE: Заблокирована учётная запись
2011,Re: Добрый день. Все тоже самое. Пишет не ве...,RE:
2012,Приложение для извлечения архива RAR Добрый д...,RE: Поддержка для открытия файла.
2013,"Изменение пароляImportance: High Коллеги, доб...",RE: не успела сменить пароль
